In [1]:
from openff.toolkit.topology import Molecule
from openff.bespokefit.workflows import BespokeWorkflowFactory
from openff.bespokefit.executor import BespokeExecutor, wait_until_complete
from openff.qcsubmit.common_structures import QCSpec
import os

In [2]:
molecule = Molecule.from_file("OA-fragment-4/OA-fragment-4.sdf")
molecule


NGLWidget()

In [3]:
host = Molecule.from_file("OA-fragment-4/OA.sdf")
host

NGLWidget()

In [4]:
workflow = BespokeWorkflowFactory(default_qc_specs=[QCSpec(method="gfn2xtb", 
                                                           basis=None, 
                                                           program="xtb", 
                                                           spec_name="gfn2xtb", 
                                                           spec_description="gfn2xtb")])
workflow.dict()

{'initial_force_field': 'openff_unconstrained-1.3.0.offxml',
 'optimizer': {'type': 'ForceBalance',
  'max_iterations': 10,
  'job_type': 'optimize',
  'penalty_type': 'L1',
  'step_convergence_threshold': 0.01,
  'objective_convergence_threshold': 0.01,
  'gradient_convergence_threshold': 0.01,
  'n_criteria': 2,
  'eigenvalue_lower_bound': 0.01,
  'finite_difference_h': 0.01,
  'penalty_additive': 1.0,
  'initial_trust_radius': -0.25,
  'minimum_trust_radius': 0.05,
  'error_tolerance': 1.0,
  'adaptive_factor': 0.2,
  'adaptive_damping': 1.0,
  'normalize_weights': False,
  'extras': {}},
 'target_templates': [{'weight': 1.0,
   'reference_data': None,
   'extras': {},
   'type': 'TorsionProfile',
   'attenuate_weights': True,
   'energy_denominator': 1.0,
   'energy_cutoff': 10.0}],
 'parameter_hyperparameters': [{'type': 'ProperTorsions',
   'priors': {'k': 6.0}}],
 'target_torsion_smirks': ['[!#1]~[!$(*#*)&!D1:1]-,=;!@[!$(*#*)&!D1:2]~[!#1]'],
 'expand_torsion_terms': True,
 'gene

In [5]:
os.environ["BEFLOW_KEEP_FILES"] = "True"
schema = workflow.optimization_schema_from_molecule(molecule=molecule)
with BespokeExecutor(
        n_fragmenter_workers=1, 
        n_optimizer_workers=1, 
        n_qc_compute_workers=3) as executor:
    opt_id = executor.submit(input_schema=schema)
    result = wait_until_complete(optimization_id=opt_id.id)

Output()

[✓] fragmentation successful

Output()

[✓] qc-generation successful

Output()

[x] optimization failed

{"type": "OSError", "message": "Source result/optimize/force-field.offxml could not be read.
 If this is a file, ensure that the path is correct.\nIf the file is present, ensure it is in
 a known SMIRNOFF encoding.\nValid formats are: ['XML']\nParsing failed with the following   
 error:\nnot well-formed (invalid token): line 1, column 6\n", "traceback": "Traceback (most 
 recent call last):\n  File \"/Users/daniellebergaziin/anaconda/envs/bespoke/lib/python3.9/si
 te-packages/celery/app/trace.py\", line 450, in trace_task\n    R = retval = fun(*args,     
 **kwargs)\n  File \"/Users/daniellebergaziin/anaconda/envs/bespoke/lib/python3.9/site-packag
 es/celery/app/trace.py\", line 731, in __protected_call__\n    return self.run(*args,       
 **kwargs)\n  File \"/Users/daniellebergaziin/anaconda/envs/bespoke/lib/python3.9/site-packag
 es/openff/bespokefit/executor/services/optimizer/worker.py\", line 29, in optimize\n        
 result = optimizer.optimize(input_schema, keep_files=settings.BEFLOW_KEEP_FILES)\n  File \"/
 Users/daniellebergaziin/anaconda/envs/bespoke/lib/python3.9/site-packages/openff/bespokefit/
 optimizers/model.py\", line 224, in optimize\n    results = cls._optimize(schema)\n  File \"
 /Users/daniellebergaziin/anaconda/envs/bespoke/lib/python3.9/site-packages/openff/bespokefit
 /optimizers/forcebalance/forcebalance.py\", line 113, in _optimize\n    results =           
 cls._collect_results(\"\", schema=schema)\n  File \"/Users/daniellebergaziin/anaconda/envs/b
 espoke/lib/python3.9/site-packages/openff/bespokefit/optimizers/forcebalance/forcebalance.py
 \", line 146, in _collect_results\n    force_field_editor =                                 
 ForceFieldEditor(results_dictionary[\"forcefield\"])\n  File \"/Users/daniellebergaziin/anac
 onda/envs/bespoke/lib/python3.9/site-packages/openff/bespokefit/utilities/smirnoff.py\",    
 line 51, in __init__\n    self.force_field = ForceField(force_field_name,                   
 allow_cosmetic_attributes=True)\n  File \"/Users/daniellebergaziin/anaconda/envs/bespoke/lib
 /python3.9/site-packages/openff/toolkit/typing/engines/smirnoff/forcefield.py\", line 353,  
 in __init__\n    self.parse_sources(sources,                                                
 allow_cosmetic_attributes=allow_cosmetic_attributes)\n  File \"/Users/daniellebergaziin/anac
 onda/envs/bespoke/lib/python3.9/site-packages/openff/toolkit/typing/engines/smirnoff/forcefi
 eld.py\", line 897, in parse_sources\n    smirnoff_data =                                   
 self.parse_smirnoff_from_source(source)\n  File \"/Users/daniellebergaziin/anaconda/envs/bes
 poke/lib/python3.9/site-packages/openff/toolkit/typing/engines/smirnoff/forcefield.py\",    
 line 1134, in parse_smirnoff_from_source\n    raise IOError(msg)\nOSError: Source           
 result/optimize/force-field.offxml could not be read. If this is a file, ensure that the    
 path is correct.\nIf the file is present, ensure it is in a known SMIRNOFF encoding.\nValid 
 formats are: ['XML']\nParsing failed with the following error:\nnot well-formed (invalid    
 token): line 1, column 6\n\n"}


worker: Warm shutdown (MainProcess)

worker: Warm shutdown (MainProcess)

worker: Warm shutdown (MainProcess)


In [ ]:
result.results.refit_parameter_values

In [ ]:
from rdkit.Chem import Draw, rdDepictor

In [ ]:
rdk_mol = molecule.to_rdkit()
rdDepictor.Compute2DCoords(rdk_mol)
matches = molecule.chemical_environment_matches(list(result.results.refit_parameter_values.keys())[5])
atoms = [m for match in matches for m in match]
Draw.MolToImage(rdk_mol, highlightAtoms=atoms)